In [150]:
import mip

In [151]:
V = [0, 1, 2, 3, 4, 5]
E = [
    [0, 1, 1, 1, 0, 0],
    [1, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
]
C = [
    [0, 2, 3, 6, 0, 0],
    [1, 0, 0, 3, 4, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 6, 0, 0, 1],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
]
start, end = 0, 5
length = range(len(V))

In [152]:
model = mip.Model()

In [153]:
x = [[model.add_var(name=f"X{j}{i}", var_type=mip.BINARY) for i in length] for j in length]

In [154]:
model.objective = mip.minimize(mip.xsum([mip.xsum([x[i][j] * C[i][j] for i in length]) for j in length]))

In [155]:
for i in length:
    for j in length:
        model.add_constr(x[i][j] <= E[i][j]) # I can only choose edges that exits

model.add_constr(mip.xsum([x[start][j] for j in length]) == 1) # only one arc exit from start
model.add_constr(mip.xsum([x[j][start] for j in length]) == 0) # no arc enter in start
model.add_constr(mip.xsum([x[j][end] for j in length]) == 1) # only one arc enter to end
model.add_constr(mip.xsum([x[end][j] for j in length]) == 0) # no arc exit from end

for i in length:
    if i == start or i == end:
        continue
    model.add_constr(mip.xsum([x[i][j] for j in length]) == mip.xsum([x[j][i] for j in length])) # what enters will also exit

In [156]:
status = model.optimize()

Cgl0002I 28 variables fixed
Cgl0004I processed model has 2 rows, 3 columns (3 integer (3 of which binary)) and 6 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 28.571%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 5
Cbc0038I Before mini branch and bound, 3 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of 5 - took 0.00 seconds
Cbc0012I Integer solution of 5 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 5, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming relaxation problem using Primal Simplex

C

In [157]:
status == mip.OptimizationStatus.OPTIMAL

True

In [158]:
model.objective.x

5.0

In [159]:
res = [[0 for i in length] for j in length]
for var in model.vars:
    res[int(var.name[1])][int(var.name[2])] = int(var.x)

print("  ", end="")
for i in length:
    print(f"{i}", end=" ")
print()
for i in length:
    print(f"{i}", end=" ")
    for j in length:
        print(res[i][j], end=" ")
    print()

  0 1 2 3 4 5 
0 0 0 1 0 0 0 
1 0 0 0 0 0 0 
2 0 0 0 1 0 0 
3 0 0 0 0 0 1 
4 0 0 0 0 0 0 
5 0 0 0 0 0 0 
